# Landsat8からLSTを取得するJupyter Notebook

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands

## 注意点

取得した画像の平均値を用いている。（NDVIの算出、


## ステップ１：GoogleEarthEngineの登録

1.1. Google Earth Engine プラットフォーム([https://earthengine.google.com/)](https://earthengine.google.com/)に移動します。

1.2. Google アカウントを使用してサインインするか、アカウントをお持ちでない場合は作成します。

1.3.プロジェクトのIDをコピーする。Pythonでプログラム作成時に用います



## ステップ２：ライブラリのインストール、GEEの認証
earthengine-apiのライブラリが必要です。
また、地図上で可視化する場合、geemapも入れておくとよいです

In [1]:
from datetime import datetime
from ipywidgets import VBox, HBox, HTML, Layout
import calendar
import ee
import geemap
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

GGE_PROJECT = 'master-research-465403'  # Google Earth EngineプロジェクトID
# Google Earth Engine APIの初期化
try:
       ee.Initialize(project=GGE_PROJECT)
except Exception as e:
       ee.Authenticate()
       ee.Initialize(project=GGE_PROJECT)

/opt/conda/lib/python3.11/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources



Successfully saved authorization token.


## ステップ３：ROIと期間、雲量の閾値の設定

ROIとは**Region of Interest**（＝注目領域、解析対象領域）の略称です。
本記事では、ベトナムのハノイのLSTを取得します。

行政区画の情報元: おすすめ
https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level1#table-schema

サンプルコード　（ハノイの行政区画を取得する）

```python
# GEEの行政境界データセット（GAUL）
# 参考：　https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level1#table-schema
gaul_level1 = ee.FeatureCollection("FAO/GAUL/2015/level1")
#ベトナムだけにフィルタ
vietnam_level1 = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Viet Nam'))
# ADM1_NAME（州名）一覧を取得
district_names = vietnam_level1.aggregate_array('ADM1_NAME').getInfo()

# 表示（重複排除＆ソート）
for name in sorted(set(district_names)):
    print(name)

ROI = ee.FeatureCollection("FAO/GAUL/2015/level1") \
         .filter(ee.Filter.eq('ADM0_NAME', 'Viet Nam')) \
         .filter(ee.Filter.eq('ADM1_NAME', 'Ha Noi City'))


In [2]:

# SHPファイルの読み込み
shp_path = '../data/SHP/研究対象領域/研究対象都市_行政区画.shp'
# 行政区画データの読み込み
boundary_df = gpd.read_file(shp_path)

hanoi_row = boundary_df[boundary_df['TinhThanh'] == 'Hà Nội'].iloc[0]
hanoi_geometry = hanoi_row.geometry
hanoi_geojson = hanoi_geometry.__geo_interface__

ROI = ee.Geometry(hanoi_geojson)

YEAR = 2025  # 対象年
MONTH = 1  # 対象月

# 雲量の閾値（%） これ以下の画像を対象とする
CLOUD_COVER = 20 

# 取得するデータの期間
START_DATE = f'{YEAR}-{MONTH:02d}-01'  # 開始日 
END_DAY = calendar.monthrange(YEAR, MONTH)[1]  # 対象月の最終日
END_DATE = f'{YEAR}-{MONTH:02d}-{END_DAY}'  # 終了日 



## ステップ4　データの取得と雲マスクの適用

### 4.1 データセットの取得

In [3]:
# Landsat 8データセット
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(START_DATE, END_DATE) \
    .filterBounds(ROI) \
    .filter(ee.Filter.lt('CLOUD_COVER', CLOUD_COVER))  # 雲量の閾値でフィルタリング

# LSTバンドを計算する関数
# 参考： https://www.usgs.gov/landsat-missions/landsat-collection-2-level-2-science-products
def compute_lst_and_reflectance(img):
    thermal_bands  = img.select(['ST_B10']).multiply(0.00341802).add(149.0)
    optical_bands = img.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(0.0000275).add(-0.2)
    return thermal_bands.addBands(optical_bands).set('system:time_start', img.get('system:time_start'))

### 4.2 画像のマスク処理
衛星画像の品質評価バンド（QA_PIXEL）を使い、特定のビットが立っているピクセル（雲・影）を除去

　参考：https://www.usgs.gov/media/files/landsat-8-9-collection-2-level-2-science-product-guide 

<details>
<summary>Landsat 8-9 QA_PIXEL ビットフラグ一覧（Table 6-2）</summary>

| ビット位置 | フラグ名              | 値の意味（0）                      | 値の意味（1）                      | 備考・詳細           |
|:----------:|:----------------------|:-----------------------------------|:-----------------------------------|:---------------------|
| 0          | Fill                  | 画像データ                         | Fillデータ                         |                      |
| 1          | 拡張クラウド          | クラウド膨張なし／クラウドなし      | クラウド膨張あり                   |                      |
| 2          | 巻雲（シーラス）      | 信頼度なし／低信頼度               | 高信頼度シーラス                   |                      |
| 3          | 雲                    | 雲信頼度が高くない                 | 雲信頼度が高い                     |                      |
| 4          | 雲影                  | 雲影信頼度が高くない               | 雲影信頼度が高い                   |                      |
| 5          | 雪                    | 雪・氷信頼度が高くない             | 雪・氷信頼度が高い                 |                      |
| 6          | 晴天                  | クラウドまたは拡張クラウドあり      | クラウド・拡張クラウドなし         |                      |
| 7          | 水                    | 陸地または雲                       | 水域                               |                      |
| 8-9        | 雲信頼度              | 信頼度なし                         | 01:低 10:中 11:高                  | 2ビット              |
| 10-11      | 雲影信頼度            | 信頼度なし                         | 01:低 11:高                        | 2ビット（10は予約）  |
| 12-13      | 雪・氷信頼度          | 信頼度なし                         | 01:低 11:高                        | 2ビット（10は予約）  |
| 14-15      | シーラス信頼度        | 信頼度なし                         | 01:低 11:高                        | 2ビット（10は予約）  |

</details>

In [4]:
def mask_clouds(img):
    qa = img.select('QA_PIXEL')
    cloud = qa.bitwiseAnd(1 << 3).eq(0)
    shadow = qa.bitwiseAnd(1 << 4).eq(0)
    mask = cloud.And(shadow)
    return img.updateMask(mask)

In [5]:
# 処理を適用
processed_dataset = dataset.map(mask_clouds).map(compute_lst_and_reflectance)


### 4.3 可視化


In [6]:
#地図上に表示
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.15,
}
CENTER= [20.99, 105.35]  # ハノイの中心座標
Map = geemap.Map(center=CENTER, zoom=8)
Map.addLayer(ROI, {}, 'ROI')
Map.add_layer(processed_dataset, visualization, 'True Color (432)')
Map

Map(center=[20.99, 105.35], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

# ステップ5 NDVI の算出

- NDVI の算出方法

 NDVI = (NIR - RED) / (NIR + RED)

- 植生指数（FV）の算出方法

 FV = ((NDVI - NDVI_min) / (NDVI_max - NDVI_min))^2

In [7]:
# NDVIの算出
def compute_ndvi(img):
    ndvi = img.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return img.addBands(ndvi)

ndvi_visualization = {
    'bands': ['NDVI'],
    'min': -1.0,
    'max': 1.0,
    'palette': ['blue', 'white', 'green']
}
ndvi_dataset = processed_dataset.map(compute_ndvi)
ndvi_mean_img = ndvi_dataset.select('NDVI').mean().clip(ROI)

Map.addLayer(ndvi_mean_img, ndvi_visualization, 'NDVI')
Map

# NDVIの最小値と最大値を算出
ndvi_stats = ndvi_mean_img.reduceRegion(
    reducer=ee.Reducer.minMax(), ## 最小値と最大値を同時に計算するメソッド
    geometry=ROI,
    scale=30,
    maxPixels=1e9
).getInfo()
print("NDVI Statistics:")
print(f"Min NDVI: {ndvi_stats['NDVI_min']}")
print(f"Max NDVI: {ndvi_stats['NDVI_max']}")




NDVI Statistics:
Min NDVI: -0.5371626615524292
Max NDVI: 0.9983720183372498


# ステップ6 植生率 FV と 放射率 EM の算出

<details> 
<summary>植生率（PV）と放射率（EM）の説明</summary>
植生率（PV）は、指定した領域内の植生の相対的な多さを、NDVI（正規化植生指数）値を解析することで定量化する指標です。
PVが高いほど、その場所に植生が多く存在することを示し、土地被覆や生態系の健全性を評価する上で重要な情報となります。

一方、放射率（EM）は地表面温度（LST）を正確に算出するために不可欠なパラメータです。

</details>

- 植生率（FV）の算出

 FV = ((NDVI - NDVI_min) / (NDVI_max - NDVI_min)) ** 2


- 放射率（EM）の算出

 EM = FV * 0.004 + 0.986


In [8]:
def add_fv_em(img, ndvi_min, ndvi_max):
    ndvi = img.select('NDVI')
    fv = ndvi.subtract(ndvi_min).divide(ndvi_max - ndvi_min).pow(2).rename(['FV'])
    em = fv.multiply(0.004).add(0.986).rename(['EM'])
    return img.addBands([fv, em])

# NDVI最小・最大値は平均画像から取得
ndvi_min = ndvi_stats['NDVI_min']
ndvi_max = ndvi_stats['NDVI_max']

# processed_dataset（ST_B10, NDVIなどを含む）にFVとEMを追加
fv_em_dataset = ndvi_dataset.map(lambda img: add_fv_em(img, ndvi_min, ndvi_max))
# 代表画像（平均画像）を作る場合は、ST_B10, EM, NDVI, FVをmean()で集計
fv_em_mean_img = fv_em_dataset.select(['ST_B10', 'EM', 'NDVI', 'FV']).mean().clip(ROI)

# ステップ7 LSTの算出

Landsat 8のLSTは Band 10を用いて算出

式：
LST = (TB / (1 + (LAMBDA * (TB / 1.438)) * ln(em))) - 273.15

In [9]:


lst_img  = fv_em_mean_img.expression(
    '(TB / (1 + (0.00115 * TB / 1.4388) * log(EM))) - 273.15',
    {
        'TB': fv_em_mean_img.select('ST_B10'),
        'EM': fv_em_mean_img.select('EM')
    }
).rename('LST')

# 可視化パラメータ例
lst_vis = {
    'min': 18,
    'max': 38,
    'palette': [
        '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
        '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
        '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
        'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
        'ff0000', 'de0101', 'c21301', 'a71001', '911003'
    ]
}

Map.addLayer(lst_img, lst_vis, 'Land Surface Temperature (LST)')

# おまけ：図のタイトル、凡例

In [10]:
minLST = 18
maxLST = 38
palette = [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003', '210300'
]
step = (maxLST - minLST) / (len(palette) - 1)
keys = [f"{(minLST + i * step):.2f}" for i in range(len(palette))]
colors = [f"#{c}" for c in palette]

# 長さを確認（デバッグ用）
assert len(keys) == len(colors), f"keys({len(keys)})とcolors({len(colors)})の長さが一致しません"

Map.add_legend(
    title="Land Surface Temperature (°C)",
    keys=keys,
    colors=colors,
    position="bottomright"
)

from ipywidgets import HTML, Layout
title_html = HTML(
    "<h2 style='text-align:center;font-weight:bold;font-size:30px;'>Land Surface Temperature - 2023 - Yogyakarta</h2>",
    layout=Layout(
        padding='20px 20px'
    )
)
display(title_html)
Map

HTML(value="<h2 style='text-align:center;font-weight:bold;font-size:30px;'>Land Surface Temperature - 2023 - Y…

Map(center=[20.99, 105.35], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

# 計算結果の表示, Geotiff形式でダウンロード

取得日、雲量、NDVIの最大値、最小値、平均値、LSTの平均値を表示します。


In [ ]:
# 取得日
# 代表画像の取得日（例：期間内の最初の画像）
first_img = processed_dataset.first()
acquisition_date = ee.Date(first_img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
print("Acquisition date:", acquisition_date)

# GeoTIFF形式でエクスポートする
task = ee.batch.Export.image.toDrive(
    image=lst_img,
    description='LST_Export',
    folder='GEE_Landsat8_LST',
    fileNamePrefix=f'LST_{YEAR}_{MONTH:02d}',
    scale=30,
    region=ROI,
    maxPixels=1e9,
    fileFormat='GeoTIFF',
    crs='EPSG:3405'
)
task.start()

Acquisition date: 2025-01-01
